# Exploring the MovieLens 1M Dataset

#### Extrated (and slightly modified) from *Python for Data Analysis (Wes McKinney)*

This dataset contains 1 million ratings collected from 6000 users on 4000 movies, and it is organized into three tables:


*   Ratings
*   Users
*   Movie information

Each table is available as a separate file, each containing a series of rows where columns are separated by *::*


[Download the dataset here](http://files.grouplens.org/datasets/movielens/ml-1m.zip)

This example illustrates a series of interesting things that we can learn from this dataset. Most operations will be performed using the pandas library. For more details, please refer to *Python for Data Analysis - page 26*.

## **Code**

Let's begin by importing pandas. It is conventional to use *pd* to denote pandas

In [1]:
import pandas as pd

Next we will import each of the three tables and assign names to each of the columns:

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')


Let's take a look at the first 5 rows of each table:

In [3]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Having all information spread across different tables makes it much more dificult to analyse the data. Using pandas's merge function, we first merge ratings with users then we merge that result with the movies data. pandas infers which columns to
use as the merge (or join) keys based on overlapping names:

In [5]:
data = pd.merge(pd.merge(ratings, users), movies)

Below is the first row in that dataset

In [6]:
data.head(1)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama


In this form, aggregating the ratings grouped by one or more user or movie characteristics is straightforward once you build some familiarity with pandas. To get mean movie ratings for each film grouped by gender, we can use the pivot_table method:


In [7]:
mean_ratings = data.pivot_table('rating', index='title', columns='gender', aggfunc='mean')

In [8]:
mean_ratings[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


If we wish to only look at movies that received more than a certain number of ratings, we can group them as follows (here using 250 ratings):

In [9]:
ratings_by_title = data.groupby('title').size()

In [10]:
ratings_by_title[:5]

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

Let's now grab the titles of movies that were rated more than 250 times:

In [11]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]

In [12]:
active_titles[:5]

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)'],
      dtype='object', name='title')

The index of titles receiving at least 250 ratings can then be used to select rows from mean_ratings above:


In [13]:
mean_ratings = mean_ratings.ix[active_titles]

In [14]:
mean_ratings[:5]

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421


To see the top films among female viewers, we can sort by the F column in descending order:

In [15]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)

In [16]:
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Likewise, for males:

In [17]:
top_male_ratings = mean_ratings.sort_values(by='M', ascending=False)

In [18]:
top_male_ratings[:10]

gender,F,M
title,,
"Godfather, The (1972)",4.314700,4.583333
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.481132,4.576628
"Shawshank Redemption, The (1994)",4.539075,4.560625
Raiders of the Lost Ark (1981),4.332168,4.520597
"Usual Suspects, The (1995)",4.513317,4.518248
Star Wars: Episode IV - A New Hope (1977),4.302937,4.495307
Schindler's List (1993),4.562602,4.491415
"Wrong Trousers, The (1993)",4.588235,4.478261
"Close Shave, A (1995)",4.644444,4.473795


Suppose you wanted to find the movies that are most divisive between male and female viewers. One way is to add a column to *mean_ratings* containing the difference in means, then sort by that:

In [19]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

Sorting by 'diff' gives us the movies with the greatest rating difference and which were preferred by women:

In [20]:
sorted_by_diff = mean_ratings.sort_values(by='diff')

In [21]:
sorted_by_diff[:10]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Reversing the order of the rows and again slicing off the top 10 rows, we get the movies preferred by men that women didn’t rate highly:


In [25]:
sorted_by_diff[::-1][:10]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


In [22]:
#P1 - aggregate of ratings for each genre
data.groupby('genres').size()

genres
Action                                           12311
Action|Adventure                                 10446
Action|Adventure|Animation                         345
Action|Adventure|Animation|Children's|Fantasy      135
Action|Adventure|Animation|Horror|Sci-Fi           618
Action|Adventure|Children's                         44
Action|Adventure|Children's|Comedy                 518
Action|Adventure|Children's|Fantasy                 44
Action|Adventure|Children's|Sci-Fi                 350
Action|Adventure|Comedy                           2077
Action|Adventure|Comedy|Crime                     2573
Action|Adventure|Comedy|Horror                     548
Action|Adventure|Comedy|Horror|Sci-Fi              945
Action|Adventure|Comedy|Romance                   5736
Action|Adventure|Comedy|Sci-Fi                    2538
Action|Adventure|Comedy|War                        213
Action|Adventure|Crime                            1862
Action|Adventure|Crime|Drama                      1431
Act

In [23]:
#P2 - top 5 highest ranked genre by women
data[data.gender == "F"].groupby('genres').size().sort_values(ascending = False)[:5]


genres
Drama             32852
Comedy            29208
Comedy|Romance    15600
Comedy|Drama      12308
Drama|Romance     11749
dtype: int64

In [24]:
#P3 - top 5 highest ranked genre by men
data[data.gender == "M"].groupby('genres').size().sort_values(ascending = False)[:5]


genres
Comedy             87675
Drama              78571
Comedy|Drama       29937
Comedy|Romance     27112
Action|Thriller    21929
dtype: int64

In [25]:
#P4 -  breakdown of a movie’s ratings by age
# assuming movie title as Toy Story (1995)
data[data.title == "Toy Story (1995)"].groupby(['rating', 'age']).size()

rating  age
1       1        2
        18       6
        25       2
        35       3
        45       1
        50       1
        56       1
2       1        6
        18      14
        25      27
        35       3
        45       5
        50       2
        56       4
3       1       25
        18      92
        25     105
        35      60
        45      25
        50      25
        56      13
4       1       45
        18     190
        25     332
        35     154
        45      59
        50      38
        56      17
5       1       34
        18     146
        25     324
        35     203
        45      53
        50      42
        56      18
dtype: int64

In [26]:
#P5 - returns a list of user_ids for other users that rated the movie identified by the provided movie_id

def same_score(user_id, movie_id):
    
    #Store the rating for the given user id from the Series object
    rating = data[data.user_id == user_id][data.movie_id == movie_id].rating.values[0]
    
    #Create the dataframe having users with same score as the given user id and movie
    frame = data.loc[(data["movie_id"] == movie_id) & (data["user_id"] != user_id) & (data["rating"] == rating)]    
    
    #Return a list of the users from the above dataframe 'frame'
    return frame['user_id'].tolist()



In [27]:
# Call the function
same_score(1,1)

/home/vinix/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[9,
 10,
 19,
 34,
 36,
 38,
 44,
 49,
 51,
 56,
 65,
 75,
 76,
 112,
 119,
 121,
 132,
 146,
 148,
 156,
 157,
 162,
 169,
 182,
 184,
 186,
 195,
 198,
 213,
 220,
 223,
 231,
 239,
 246,
 258,
 272,
 273,
 284,
 300,
 301,
 307,
 325,
 333,
 343,
 346,
 350,
 369,
 385,
 389,
 411,
 413,
 429,
 434,
 438,
 451,
 456,
 463,
 474,
 478,
 479,
 496,
 523,
 524,
 528,
 529,
 531,
 536,
 541,
 543,
 549,
 550,
 555,
 563,
 577,
 583,
 590,
 605,
 606,
 610,
 613,
 623,
 626,
 634,
 635,
 637,
 641,
 645,
 646,
 669,
 676,
 678,
 690,
 692,
 707,
 710,
 714,
 716,
 718,
 722,
 726,
 729,
 731,
 733,
 739,
 749,
 752,
 753,
 765,
 776,
 791,
 801,
 802,
 813,
 817,
 822,
 839,
 840,
 858,
 869,
 877,
 880,
 885,
 890,
 891,
 894,
 926,
 929,
 933,
 934,
 938,
 946,
 949,
 951,
 957,
 1000,
 1004,
 1015,
 1021,
 1032,
 1036,
 1051,
 1054,
 1055,
 1057,
 1059,
 1061,
 1072,
 1076,
 1087,
 1107,
 1112,
 1117,
 1124,
 1125,
 1132,
 1141,
 1145,
 1155,
 1183,
 1202,
 1203,
 1221,
 1223,
 1224,


In [35]:
#P6 - A statistical plot function which accepts any movie_id and displays the popularity among all genders

from pylab import *

def plot_by_gender(movie_id):
    
    #Fetch the dataframe by grouping on gender based on the given movie id
    df = data[data.movie_id == movie_id].groupby('gender').size()
    
    #Plot
    # make a square figure and axes
    figure(1, figsize=(6,6))
    
    # The slices will be ordered and plotted counter-clockwise.
    labels = 'Female', 'Male'
    fracs = [df.values[0], df.values[1]]
    
    pie(fracs, labels=labels, autopct='%.1f%%')
    
    title('Popularity based on gender', bbox={'facecolor':'0.8', 'pad':5})
    
    show()
    
    return


#### Observation:

This feature or statistics gives us an idea like if a movie is a girl or a boy - acceptable. A critical observation can be made by psychologists or therapists on the reactions of girls to boys on any movie or genre to study human emotions or cognitive science and use them in treating mental disorders.
 

In [37]:
# Call plot function
plot_by_gender(8)